Imports for Inferencing

In [1]:
import tensorflow as tf
from keras.applications.xception import Xception as net
from keras_preprocessing.image.utils import img_to_array,load_img, array_to_img
from keras.applications.resnet_v2 import preprocess_input, decode_predictions
import numpy as np
import cv2
from tensorflow.python.keras import backend as K
import pandas as pd

2022-06-28 18:07:45.391784: I tensorflow/core/util/util.cc:169] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.


Imports for dataset aquisition from preprocessed data

In [2]:
import os
import sys
import yaml

p = os.path.abspath('.')
sys.path.insert(1, p)

In [ ]:
from data_import_and_preprocessing.dataset_formation import DataParser, ImageDataExtractor, LabelExtractor, \
    DataSetCreator

print(os.getcwd())
os.chdir('/mnt/0A60B2CB60B2BD2F/Projects/leo_flow_regime_recognition')
print(os.getcwd())
with open('params.yaml', 'r') as stream:
    params = yaml.safe_load(stream)
no_epochs = params['training']['no_epochs']
batch_size = params['training']['batch_size']
picture_width = params['preprocessing']['picture_width']
picture_hight = params['preprocessing']['picture_hight']
no_classes = params['preprocessing']['no_classes']

data_dir = 'data/preprocessed'
data_parser = DataParser(data_dir)
image_data_extractor = ImageDataExtractor((picture_width, picture_hight, 1))
label_extractor = LabelExtractor(no_classes=no_classes) #anzahl der classen
dataset = DataSetCreator(data_parser, image_data_extractor, label_extractor, no_repeats=no_epochs)
pathList = []
for imagePath in dataset.data_points:
    pathList.append(imagePath.path_to_data)


In [ ]:
pathlist. get a picture
path = '/mnt/0A60B2CB60B2BD2F/Projects/leo_flow_regime_recognition/data/preprocessed/1100000001.png'
finde den namen "1100000001" mit dem neuen frame mit allen datapoints. ersetze path damit
img = cv2.imread(path)

In [ ]:
img_preprocessed = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
img_preprocessed = cv2.resize(img_preprocessed, (32, 32), interpolation=cv2.INTER_CUBIC)
img_preprocessed  = img_preprocessed / 255

x = np.expand_dims(img_preprocessed, axis=0)

In [ ]:
model_path = "/mnt/0A60B2CB60B2BD2F/Projects/leo_flow_regime_recognition/training/model/trained_model.h5"
reconstructed_model = tf.keras.models.load_model(model_path)

In [ ]:
preds = reconstructed_model.predict(x)

In [ ]:
# decode the results into a list of tuples (class, description, probability)
preds

In [ ]:
pred=np.around(preds,3)

In [ ]:
i=0
inferedClass=""
classaccuracy=0

inferedAccuracyFlooded=0
inferedAccuracyLoaded=0
inferedAccuracyDispersed=0

for v in pred[0]:
    if i==0:
        inferedAccuracyFlooded = v
    if i==1:
        inferedAccuracyLoaded = v
    if i==2:
        inferedAccuracyDispersed = v
    i=i+1
classaccuracy = max(inferedAccuracyFlooded,inferedAccuracyDispersed,inferedAccuracyLoaded)
if classaccuracy == inferedAccuracyFlooded:
    inferedClass= "Flooded"
if classaccuracy == inferedAccuracyLoaded:
    inferedClass= "Loaded"
if classaccuracy == inferedAccuracyDispersed:
    inferedClass= "Despersed"
print(inferedClass+": "+str(classaccuracy))

In [ ]:
print(reconstructed_model.layers[11])

In [ ]:

# lstm_67 is the second layer.
dense2D = K.function([reconstructed_model.layers[0].input], [reconstructed_model.layers[11].output])
dense2D = dense2D(x)[0]
dense2D

import dataset

In [ ]:
from report import grad_cam
for idx in range(len(reconstructed_model.layers)):
  print(reconstructed_model.get_layer(index = idx).name)

In [ ]:
reconstructed_model.summary()

In [ ]:
heatmap = grad_cam.get_gradcam_heatmap(img_preprocessed,reconstructed_model, "conv2d_1")

"Delete all photos in saliencemap"

In [ ]:
import os, shutil
folder = '/mnt/0A60B2CB60B2BD2F/Projects/leo_flow_regime_recognition/data/saliencemap'
for filename in os.listdir(folder):
    file_path = os.path.join(folder, filename)
    try:
        if os.path.isfile(file_path) or os.path.islink(file_path):
            os.unlink(file_path)
        elif os.path.isdir(file_path):
            shutil.rmtree(file_path)
    except Exception as e:
        print('Failed to delete %s. Reason: %s' % (file_path, e))

"Mark calculate gradcam"

In [ ]:
alphaIncrement=0.0
j=1
jIncrement=1
heatmapType="cool"
for i in range(1):
    alpha=0.9975
    for k in range(1):
        salienceMap = grad_cam.combine_image_and_heatmap(img,heatmap,alpha,heatmapType)
        picture= array_to_img(salienceMap)
        picture.save("/mnt/0A60B2CB60B2BD2F/Projects/leo_flow_regime_recognition/data/saliencemap/"+heatmapType+"_alpha"+str(alpha)+"_"+inferedClass+str(round(classaccuracy*1000))+".png")
        alpha=alpha+alphaIncrement
    j=j+jIncrement